# Importance12 Districts: distributionGO_distmonthProcess

## Importance12 Districts: distributionGO_distmonthProcess

- Step 1: List all files and check if empty. Subset based on emptyness
- Step 2: Read file and add file name, split file name to additional columns and add column with path
- Step 3: Define variables and if necessary to calculate relative values
- Step 4: Determine how to sumarize columns if possible to sum or average and do so
- Step 5: Break data by years
- Step 6: Summarize data by zilas. Keep numeric variables, division, zila and year
- Step 7: Create file with zilas, division and 
- Step 8: Write everything to the output files

## Paths

In [56]:
DATA = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/dgfp/data/importance12_districts/distributionGO_distmonthProcess/'
GEO = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/dgfp/geos/dgfp_geo.csv'
OUT = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/dgfp/data/'

## Packages

In [57]:
import os
import re
import sys
import glob
import numpy as np
import pandas as pd

## Custom functions

In [58]:
def list_files(path):
    files_list = []
    for path, subdirs, files in os.walk(path):
        for file in files:
            if bool(re.search(pattern=r'.csv', string=file)):
                files_list.append(os.path.join(path, file))
    return files_list
    
def read_arrange_files(files_list):
    data_list = {}
    log_columns = ['empty', 'full_name_path', 'file_name1', 'file_name2',
                   'year', 'month', 'upazila', 'division', 'upazila_full']
    log_array = []
    for file in files_list:
        tmp = pd.read_csv(file, sep='\t')
        tmp['Month'] = tmp['monthid'].str.split(pat=",", expand=True)[0]
        tmp['Year'] = tmp['monthid'].str.split(pat=",", expand=True)[1]
        log_list = []
        log_list.append(tmp.empty)
        if tmp.empty:
            log_list.append(file)
            log_list.append(os.path.split(file)[1])
            name = os.path.split(file)[1]
            name = name.replace(".csv", "")
            log_list.append(name)
            name_split = name.split("-")
            log_list.append(name_split[0])
            log_list.append(name_split[1])
            log_list.append(name_split[3])
            log_list.append(name_split[4])
            log_list.append(name_split[4]+name_split[3])
        else:
            tmp['full_name_path'] = file
            log_list.append(file)
            tmp['file_name1'] = os.path.split(file)[1]
            log_list.append(os.path.split(file)[1])
            name = os.path.split(file)[1]
            name = name.replace(".csv", "")
            tmp['file_name2'] = name
            log_list.append(name)
            name_split = name.split("-")
            tmp['year'] = int(name_split[0])
            log_list.append(name_split[0])
            tmp['month'] = int(name_split[1])
            log_list.append(name_split[1])
            tmp['upazila'] = name_split[3]
            log_list.append(name_split[3])
            tmp['division'] = name_split[4]
            log_list.append(name_split[4])
            tmp['upazila_full'] = name_split[4]+name_split[3]
            log_list.append(name_split[4]+name_split[3])
            data_list[name] = tmp
        log_array.append(log_list)
    return data_list, pd.DataFrame(log_array, columns=log_columns)

def concat_data(ddict):
    out_list = []
    for key, value in ddict.items():
        out_list.append(value)
    df = pd.concat(out_list, axis=0)
    return df

def break_by_years(input_df, year_var):
    df = input_df.copy(deep=True)
    out_dict = {}
    for year in df[year_var].unique():
        out_dict[str(year)] = df[df[year_var] == year]
    return out_dict

def summarize_zilas(data_dict):
    out_dict = {}
    for key, value in data_dict.items():
        tmp = value.copy(deep=True)
        subset_vars = ['upazila', 'division', 'Shukhi', 'Apon', 'Pill_total', 'Condom',
                       'Inj_vial', 'Inj_siringe','IUD_normal', 'IUD_partum',
                       'IUD_total', 'IUD_remove', 'Imp_normal', 'Imp_Jadel',
                       'Imp_total', 'implant_remove', 'ecp', 'misoprostol', 
                       'mrm_pack', 'inj_mgs4', 'mnp_saset', 'ifa_number',
                       'sanitary_pad', 'per_male', 'femalenormal', 'femalepn',
                       'female_total', 'Permanent_method']        
        tmp = tmp[subset_vars]
        tmp = tmp.groupby('upazila').sum()
        tmp = tmp.reset_index()
        out_dict[key] = tmp
    return out_dict

def update_geos(datad, geo_df):
    out = {}
    for key, value in datad.items():
        tmp = geo_df.merge(value, how='left', left_on='upazila_dgfp', right_on='upazila')
        out[key] = tmp
    return out


def clean_up(data_dict):
    out = {}
    for key, value in data_dict.items():
        tmp = value
        tmp['geo'] = tmp['division_geo'].str.cat(tmp['zila_geo'], sep="")
        tmp = value.drop(['upazila_dgfp', 'division_dgfp', 'upazila_full_dgfp', 'division_geo',
                          'division', 'zila_geo','zila','upazila'], axis=1)
        out[key] = tmp
    return out

## Data reading and processing 

In [59]:
files = list_files(path=DATA) 
files

['/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/dgfp/data/importance12_districts/distributionGO_distmonthProcess/2018-12-28-0305-03.csv',
 '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/dgfp/data/importance12_districts/distributionGO_distmonthProcess/2018-12-28-0114-01.csv',
 '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/dgfp/data/importance12_districts/distributionGO_distmonthProcess/2018-12-28-0209-02.csv',
 '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/dgfp/data/importance12_districts/distributionGO_distmonthProcess/2018-12-28-0205-02.csv',
 '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/dgfp/data/importance12_districts/distributionGO_distmonthProcess/2018-12-28-0108-07.csv',
 '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/dgfp/data/importance12_districts/distributionGO_distmonthProcess/2018-12-28-0411-04.csv',
 '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/dgfp/data/importance12_districts/distributionGO

In [60]:
data, log = read_arrange_files(files_list=files)

### Log data

In [61]:
log['empty'].value_counts()
log_true = log[log['empty'] == True]
display(log_true.head())

,empty,full_name_path,file_name1,file_name2,year,month,upazila,division,upazila_full


## Missing data by years

In [62]:
print(log_true['year'].value_counts())

Series([], Name: year, dtype: int64)


## Missing data by months

In [63]:
print(log_true['month'].value_counts())

Series([], Name: month, dtype: int64)


## Missing data by divisions

In [64]:
print(log_true['division'].value_counts())

Series([], Name: division, dtype: int64)


In [65]:
data_df = concat_data(ddict=data)
print(data_df.shape)

(8672, 37)


In [66]:
display(data_df.head())
data_df.columns

,monthid,Shukhi,Apon,Pill_total,Condom,Inj_vial,Inj_siringe,IUD_normal,IUD_partum,IUD_total,...,Month,Year,full_name_path,file_name1,file_name2,year,month,upazila,division,upazila_full
0,"July,2007",91661,0,91661,52581,24197,0,136,0,136,...,July,2007,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305
1,"August,2007",84221,0,84221,46020,18188,0,87,0,87,...,August,2007,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305
2,"September,2007",84714,0,84714,56346,20243,0,108,0,108,...,September,2007,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305
3,"October,2007",81598,0,81598,43630,23969,0,133,0,133,...,October,2007,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305
4,"November,2007",84763,0,84763,50118,22869,0,434,0,434,...,November,2007,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305


Index(['monthid', 'Shukhi', 'Apon', 'Pill_total', 'Condom', 'Inj_vial',
       'Inj_siringe', 'IUD_normal', 'IUD_partum', 'IUD_total', 'IUD_remove',
       'Imp_normal', 'Imp_Jadel', 'Imp_total', 'implant_remove', 'ecp',
       'misoprostol', 'mrm_pack', 'inj_mgs4', 'mnp_saset', 'ifa_number',
       'sanitary_pad', 'per_male', 'femalenormal', 'femalepn', 'female_total',
       'Permanent_method', 'Month', 'Year', 'full_name_path', 'file_name1',
       'file_name2', 'year', 'month', 'upazila', 'division', 'upazila_full'],
      dtype='object')

## Break data by years

In [67]:
datad = break_by_years(input_df=data_df, year_var='Year')
datad.keys()

dict_keys(['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'])

In [68]:
datad['2007'].head()

,monthid,Shukhi,Apon,Pill_total,Condom,Inj_vial,Inj_siringe,IUD_normal,IUD_partum,IUD_total,...,Month,Year,full_name_path,file_name1,file_name2,year,month,upazila,division,upazila_full
0,"July,2007",91661,0,91661,52581,24197,0,136,0,136,...,July,2007,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305
1,"August,2007",84221,0,84221,46020,18188,0,87,0,87,...,August,2007,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305
2,"September,2007",84714,0,84714,56346,20243,0,108,0,108,...,September,2007,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305
3,"October,2007",81598,0,81598,43630,23969,0,133,0,133,...,October,2007,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305
4,"November,2007",84763,0,84763,50118,22869,0,434,0,434,...,November,2007,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305


## Summarize data 

- **Variables**:
    - **monthid:** Month,Year 
    - **Shukhi:** Distribution_Oral Pill (Cycle) Shukhi
    - **Apon:** Distribution_Oral Pill (Cycle) Apon
    - **Pill_total:** Distribution_Oral Pill (Cycle) Total
    - **Condom:** Distribution Condom (Nirapad) (Pieces)
    - **Inj_vial:** Distribution Injectable Vial
    - **Inj_siringe:** Distribution Injectable Syringe
    - **IUD_normal:** Distribution intra-uterine devices new intra-uterine normal
    - **IUD_partum:** Distribution intra-uterine new intra-uterine postpartum
    - **IUD_total:** Distribution intra-uterine new intra-uterine Total
    - **IUD_remove:** Distribution intra-uterine remove
    - **Imp_normal:** Distribution implant eew iumplanon
    - **Imp_Jadel:** Distribution implant eew jadel
    - **Imp_total:** Distribution implant eew total
    - **implant_remove:** Distribution implant remove
    - **ecp:** Distribution emergency contraceptive pills (ECP)
    - **misoprostol:** Distribution Misoprostol (Dose)
    - **mrm_pack:** Distribution MRM (Pack)
    - **inj_mgs4:** Distribution injection MgS04
    - **mnp_saset:** Distribution MNP (Saset)
    - **ifa_number:** Distribution IFO (Number)
    - **sanitary_pad:** Distribution Sanitary Pad (Number)
    - **per_male:** Performed Permanant Method Male (Number)
    - **femalenormal:** Performed Permanant Method Female (Number) Normal
    - **femalepn:** Performed Permanant Method Female (Number) Postpartum
    - **female_total:** Performed Permanant Method Female (Number) Total
    - **Permanent_method:** Performed Permanant Method Total
    - **Month:** Month
    - **Year:** Year
    - **full_name_path:** Full name with path
    - **file_name1:** File name with extension 
    - **file_name2:** File name without extension
    - **year:** Year
    - **month:** Month
    - **upazila:** Upazila DGFP code
    - **division:** Division  DGFP code
    - **upazila_full:** Full upazila DGFP code

![Variables](dp_dgfp_imp12distr_distributionGO_distmonthProcess.jpeg)

- Indicators:
    - All variables divided by population of women 15-45 years of old

In [69]:
data = summarize_zilas(data_dict=datad)

## Read geographies

In [70]:
dgfp_geo = pd.read_csv(GEO)
dgfp_geo['upazila_dgfp'] = dgfp_geo['upazila_dgfp'].astype(str).str.pad(width=4, side='left', fillchar='0')
dgfp_geo['division_dgfp'] = dgfp_geo['division_dgfp'].astype(str).str.pad(width=2, side='left', fillchar='0')
dgfp_geo['upazila_full_dgfp'] = dgfp_geo['upazila_full_dgfp'].astype(str).str.pad(width=6, side='left', fillchar='0')
dgfp_geo['division_geo'] = dgfp_geo['division_geo'].astype(str).str.pad(width=2, side='left', fillchar='0')
dgfp_geo['zila_geo'] = dgfp_geo['zila_geo'].astype(str).str.pad(width=2, side='left', fillchar='0')
dgfp_geo = dgfp_geo.drop(['thana_dgfp', 'Unnamed: 8'], axis=1)
print(dgfp_geo.shape)
dgfp_geo.head()
dgfp_geo = dgfp_geo.drop_duplicates()
print(dgfp_geo.shape)

(29260, 7)
(64, 7)


## Update geographies

In [71]:
data = update_geos(datad=data, geo_df=dgfp_geo)

## Clean up before writing data

In [72]:
data = clean_up(data_dict=data)

In [73]:
data['2007']

,Shukhi,Apon,Pill_total,Condom,Inj_vial,Inj_siringe,IUD_normal,IUD_partum,IUD_total,IUD_remove,...,inj_mgs4,mnp_saset,ifa_number,sanitary_pad,per_male,femalenormal,femalepn,female_total,Permanent_method,geo
0,459254,0,459254,199439,43476,0,882,0,882,0,...,0,0,0,0,636,210,0,210,846,5577
1,756828,0,756828,463602,59461,0,1144,0,1144,0,...,0,0,0,0,62,1027,0,1027,1089,4044
2,703282,0,703282,659337,53477,0,1250,0,1250,0,...,0,0,0,0,1003,566,0,566,1569,4001
3,2245919,0,2245919,1106026,83882,0,5311,0,5311,0,...,0,0,0,0,3342,922,0,922,4264,3061
4,366220,0,366220,228962,32403,0,978,0,978,0,...,0,0,0,0,277,113,0,113,390,1079
5,1386072,0,1386072,669371,99858,0,11671,0,11671,0,...,0,0,0,0,2949,2438,0,2438,5387,5527
6,1514627,0,1514627,2277513,249604,0,6349,0,6349,0,...,0,0,0,0,4442,1548,0,1548,5990,3026
7,524763,0,524763,352815,61434,0,1354,0,1354,0,...,0,0,0,0,796,299,0,299,1095,2022
8,574801,0,574801,755098,45654,0,2588,0,2588,0,...,0,0,0,0,512,1173,0,1173,1685,3029
9,1373742,0,1373742,1169886,107700,0,2800,0,2800,0,...,0,0,0,0,1685,1443,0,1443,3128,5010


## Writing data

In [74]:
def write_out(data_dict, out_dir):
    out = {}
    for key, value in data_dict.items():
        var_names = ["distr_GO" + var if var != 'geo' else var for var in value.columns]
        value.columns = var_names
        value.to_csv(path_or_buf=os.path.join(out_dir, "data_dgfp_imp12distr_distributionGO_distmonthProcess_" + str(key) + ".csv"), 
                    index=False, index_label=False)
write_out(data_dict=data, out_dir=OUT)